In [13]:
from nltk import word_tokenize
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import nltk
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import sklearn.metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import grid_search
from sklearn.linear_model import LogisticRegression
import json
from sklearn.base import BaseEstimator, TransformerMixin
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

In [17]:
class ExtractRecipe():
    """ 
    Extracts recipe information from JSON.
    """
    def __init__(self, json):
        self.recipe_id = self.set_id(json)
        self.cuisine = self.set_cuisine(json)
        self.ingredients = self.set_ingredients(json)
        self.ingredient_count = len(self.ingredients)
        
    def __str__(self):
        return "ID: %s\nCuisine: %s\nIngredients: %s\nNumber of Ingredients: %s" % (self.recipe_id,
                                    self.cuisine,', '.join(self.ingredients),self.ingredient_count)
    def set_id(self,json):
        """
        sets the recipe id.
        """
        try:
            return json['id']
        except KeyError:
            return '-99'
        
    def set_cuisine(self,json):
        """
        sets the recipe cuisine.
        """
        try:
            return json['cuisine']    
        except KeyError:
            return ''
        
    def set_ingredients(self,json):
        """
        sets the recipe ingredients.
        """
        try:
            return json['ingredients']
        except KeyError:
            return []
        
    def clean_ingredient(self,s):
        """
        returns a cleaned up version of the entered ingredient.
        """
        from re import sub
        return sub('[^A-Za-z0-9]+', ' ', s)
    
    def get_train(self):
        """
        returns a dictionary of data for the training set.
        """
        return {
            'cuisine':self.cuisine,
            'ingredients':', '.join([self.clean_ingredient(x) for x in self.ingredients]),
            'ingredient_count':self.ingredient_count
        }
    
    def get_predict(self):
        """
        returns a dictionary of data for predicting recipes.
        """
        return {
            'id':self.recipe_id,
            'ingredients':', '.join([self.clean_ingredient(x) for x in self.ingredients]),
            'ingredient_count':self.ingredient_count
        }   


def loadTrainSet(dir='train.json'):
    """
    Read in JSON to create training set.
    """
    import json
    from pandas import DataFrame, Series
    from sklearn.preprocessing import LabelEncoder
    X = DataFrame([ExtractRecipe(x).get_train() for x in json.load(open(dir,'rb'))])
    encoder = LabelEncoder()
    X['cuisine'] = encoder.fit_transform(X['cuisine'])
    return X, encoder

def loadTestSet(dir='test.json'):
    """
    Read in JSON to create test set.
    """
    import json
    from pandas import DataFrame
    return DataFrame([ExtractRecipe(x).get_predict() for x in json.load(open(dir,'rb'))])     

In [135]:
train_df, encoder = loadTrainSet()

import nltk

def make_unique_ingredients_dict(df):
    ingredients = dict()
    stemmer = nltk.PorterStemmer()
    i = 0

    for row in df.itertuples():
        ingredients_list = row[3].split(', ')

        for ingredient in ingredients_list:
            words = ingredient.split()
            stemmed_words = []

            for word in words:
                stemmed_words.append(stemmer.stem(word.lower()))

            stemmed_ingredient = ' '.join(stemmed_words)

            if stemmed_ingredient not in ingredients:
                ingredients[stemmed_ingredient] = i
                i = i + 1
    
    return ingredients

In [189]:
import scipy as sp
from scipy.sparse import csr_matrix

from sklearn import cross_validation

def estimate_acc(clf, train_data, train_labels):
    scores = cross_validation.cross_val_score(clf, train_data, train_labels, cv = 5)

    print('Accuracy on training set: {0} +/- {1}'.format(scores.mean(), scores.std() * 2))
    
def write_submission(reasult_dict):
    """File to upload"""
    writer = csv.writer(open('submission.csv', 'wt'))
    writer.writerow(['id','cuisine'])
    for key, value in result_dict.items():
        writer.writerow([key, value])

def make_preprocessed_matrix(df, unique_ingridients):
    """Stemm ingredients in dataframe and put into X"""
    
    X = sp.sparse.csr_matrix((train_df.shape[0], 1 + len(unique_ingridients)))
    
    stemmer = nltk.PorterStemmer()
    #print df
    for example_number, row in enumerate(df.itertuples()):
        #print row[3], '\n'
        #ingridients_list = row[3].split(', ')
        
        for ingridient in ingridients_list:
            words = ingridient.split()
            stemmed_words = []

            for word in words:
                stemmed_words.append(stemmer.stem(word.lower()))
            #print stemmed_words
            stemmed_ingridient = ' '.join(stemmed_words)
            
            ingridient_index = unique_ingridients[stemmed_ingridient]
            
            X[example_number, ingridient_index] = 1
            #print X
        
        X[example_number, len(unique_ingridients) - 1] = row[2]
        
    return X
            


In [190]:
a = train_df[0:2]
#print a
unique_ingredients = make_unique_ingredients_dict(a)
#print unique_ingredients
print train_df['ingredients'][0], train_df['ingredients'][1]
ingredients = train_df['ingredients']

X = make_preprocessed_matrix(train_df, unique_ingredients)
y = train_df['cuisine']
print len(y), X.shape, len(ingredients)

romaine lettuce, black olives, grape tomatoes, garlic, pepper, purple onion, seasoning, garbanzo beans, feta cheese crumbles plain flour, ground pepper, salt, tomatoes, ground black pepper, thyme, eggs, green tomatoes, yellow corn meal, milk, vegetable oil


NameError: global name 'ingridients_list' is not defined

In [56]:
from sklearn.naive_bayes import BernoulliNB

clf = BernoulliNB(alpha = 0, fit_prior = False)
estimate_acc(clf, X, y)

Accuracy on training set: 0.617489236593 +/- 0.0109575034339


In [109]:
test_df = loadTestSet()

ingredients_test = test_df['ingredients'] for item in test_df]
X_test = dok_matrix((len(ingredients_test), len(unique_ingredients)), dtype=np.dtype(bool))

for d,dish in enumerate(ingredients_test):
    for i,ingredient in enumerate(unique_ingredients):
        if ingredient in dish:
            X_test[d,i] = True

y_predicted = clf.predict(X_test)

cuisines_predicted = encoder.inverse_transform(y_predicted)
result_dict = dict(zip(test_df['id'], cuisines_predicted))

SyntaxError: invalid syntax (<ipython-input-109-95ff17a93931>, line 3)